In [1]:
import openai
from dotenv import load_dotenv
import os
from IPython.display import display, HTML, JSON, Markdown
from langchain_openai import AzureChatOpenAI

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_DEPLOYMENT_ENDPOINT = os.getenv("OPENAI_DEPLOYMENT_ENDPOINT")
OPENAI_GPT35_DEPLOYMENT_NAME = os.getenv("OPENAI_GPT35_DEPLOYMENT_NAME")


In [2]:
def init_llm(model=OPENAI_GPT35_DEPLOYMENT_NAME,
             deployment_name=OPENAI_GPT35_DEPLOYMENT_NAME,
             openai_api_version="2024-02-15-preview",
             temperature=0,
             max_tokens=400
             ):

    llm = AzureChatOpenAI(deployment_name=deployment_name,
                            model=model,
                            openai_api_version=openai_api_version,
                            azure_endpoint=OPENAI_DEPLOYMENT_ENDPOINT,
                            temperature=temperature,
                            max_tokens=max_tokens
                            )
    return llm

llm = init_llm()

In [3]:
from langchain import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.document_loaders import PyPDFLoader

fileName = "./data/Best-Practices-in-Change-Management-Full-Report-Digital-11th-Edition.pdf"
loader = PyPDFLoader(fileName)
pages = loader.load_and_split()
print("Number of pages: ", len(pages))

Number of pages:  367


![Upload](./images/map_reduce.png)

In [4]:
map_prompt_template = """
                      Write a summary of this chunk of text that includes the main points and any important details.
                      {text}
                      """

map_prompt = PromptTemplate(template=map_prompt_template, input_variables=["text"])

combine_prompt_template = """
                      Write a concise summary of the following text delimited by triple backquotes.
                      Return your response in bullet points which covers the key points of the text.
                      ```{text}```
                      BULLET POINT SUMMARY:
                      """

combine_prompt = PromptTemplate(
    template=combine_prompt_template, input_variables=["text"]
)

In [5]:
map_reduce_chain = load_summarize_chain(
    llm,
    chain_type="map_reduce",
    map_prompt=map_prompt,
    combine_prompt=combine_prompt,
    return_intermediate_steps=True,
)

In [6]:
# This might take some time to run. For demo purposes, limit it to 10 pages.
map_reduce_outputs = map_reduce_chain.invoke({"input_documents": pages[100:110]})
print("Number of outputs: ", len(map_reduce_outputs))

Number of outputs:  3


In [7]:
display(Markdown(map_reduce_outputs['output_text']))

- Participants in a study used various approaches to demonstrate the value of applying change management for a project.
- The most common approaches used to demonstrate value-add were increased realization of project benefits, improved adoption of the change, and increased engagement and involvement.
- Participants measured the overall outcome of applying change management through impacted employee responses, project results, and project execution.
- Surveys, level of adoption/observation, feedback, assessments, eliminated challenges, and comparisons to other projects were used to measure change management effectiveness.
- Channels used to demonstrate the value-add of applying change management included employee surveys, anecdotal evidence, presentations, industry data, and lessons-learned reporting.
- 51% of participants in the 2019 study reported measuring change management effectiveness.
- 44% of participants measured whether change was occurring at the individual level.
- The most commonly used metrics for individual change measurement were compliance and usage.
- Observation, communication, tests and assessments, performance evaluations, existing business metrics, and surveys were used to measure individual progress and effectiveness during a change.
- 24% of participants measured the speed of adoption in change management programs.
- Effective change management programs were found to improve speed of adoption, ultimate utilization, and proficiency.
- Feedback, observation, surveys, assessments, performance evaluations, compliance reports, audits, error tracking, and issues raised were used to collect feedback and measure proficiency and ultimate utilization in change management.
- Adherence to the schedule and staying within budget were the most frequently cited indicators for project success.
- Obstacles in different areas of a project include leadership and sponsorship, project management, change management, user response, achievement of interim milestones, error tracking, and engagement measures.
- Applying a structured change management methodology and starting the application early in the project lifecycle are important factors for project success.

In [8]:
import pandas as pd
from pathlib import Path as p
final_mp_data = []
for doc, out in zip(map_reduce_outputs["input_documents"], map_reduce_outputs["intermediate_steps"]):
    output = {}
    output["file_name"] = p(doc.metadata["source"]).stem
    output["file_type"] = p(doc.metadata["source"]).suffix
    output["page_number"] = doc.metadata["page"]
    output["chunks"] = doc.page_content
    output["concise_summary"] = out
    final_mp_data.append(output)

pdf_mp_summary = pd.DataFrame.from_dict(final_mp_data)
pdf_mp_summary = pdf_mp_summary.sort_values(
    by=["file_name", "page_number"]
)  # sorting the dataframe by filename and page_number
pdf_mp_summary.reset_index(inplace=True, drop=True)
pdf_mp_summary.head()

,file_name,file_type,page_number,chunks,concise_summary
0,Best-Practices-in-Change-Management-Full-Repor...,.pdf,95,Prosci ® Best Practices in Change Managemen...,The main points of this text are:\n\n- Partici...
1,Best-Practices-in-Change-Management-Full-Repor...,.pdf,96,96 • © 2020 Prosci Inc. Figure 5.9 – Measure...,This chunk of text discusses the methods used ...
2,Best-Practices-in-Change-Management-Full-Repor...,.pdf,97,Prosci ® Best Practices in Change Managemen...,This text discusses the best practices in chan...
3,Best-Practices-in-Change-Management-Full-Repor...,.pdf,98,98 • © 2020 Prosci Inc. 2. Observation and c...,This chunk of text discusses various methods f...
4,Best-Practices-in-Change-Management-Full-Repor...,.pdf,99,Prosci ® Best Practices in Change Managemen...,This text discusses the findings of a study on...


In [9]:
index = 6
print("[Context]")
print(pdf_mp_summary["chunks"].iloc[index])
print("\n\n [Simple Summary]")
print(pdf_mp_summary["concise_summary"].iloc[index])
print("\n\n [Page number]")
print(pdf_mp_summary["page_number"].iloc[index])
print("\n\n [Source: file_name]")
print(pdf_mp_summary["file_name"].iloc[index])

[Context]
Prosci ®    Best Practices in Change Management© 2020 Prosci Inc.  •  1013. Feedback and observation  
Participants collected feedback through interviews, 
focus groups and one-on-one conversations. 
Feedback was collected from key individuals 
including employees, managers, clients and change 
network members. Participants also gauged ultimate 
use by observing and monitoring employees in the 
new state.
4. Work output and performance Identified less than half as often as the previous 
finding, employee performance and output also 
determined ultimate use. Performance evaluations, 
project goals and updated key performance 
indicators (KPIs) were used. The quality and quantity 
of work output was also evaluated to determine how 
many employees were using the solution.
5. Timeline and process completion Participants tracked timeline, process and task 
completion often using checklists. Some said the time 
it took for employees to become efficient using a new 
solution contrib